In [29]:
! pip install happytransformer

     ---------------------------------------- 45.5/45.5 kB 2.2 MB/s eta 0:00:00
     -------------------------------------- 422.5/422.5 kB 6.5 MB/s eta 0:00:00
     -------------------------------------- 977.6/977.6 kB 5.6 MB/s eta 0:00:00
     -------------------------------------- 474.6/474.6 kB 7.4 MB/s eta 0:00:00
     -------------------------------------- 323.6/323.6 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 21.5/21.5 MB 4.7 MB/s eta 0:00:00
     -------------------------------------- 132.9/132.9 kB 7.7 MB/s eta 0:00:00
     ---------------------------------------- 61.7/61.7 kB ? eta 0:00:00
     ---------------------------------------- 110.5/110.5 kB ? eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.4
    Uninstalling dill-0.3.4:
      Successfully uninstalled dill-0.3.4


In [30]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline, DistilBertTokenizer, DistilBertForSequenceClassification
import torch
from happytransformer import HappyTextClassification 

In [15]:
sentimentRecord = "Sucks to suck loser!"

In [16]:
# To classify emotion
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

In [17]:
emotion_classifier(sentimentRecord)

[[{'label': 'anger', 'score': 0.371420294046402},
  {'label': 'disgust', 'score': 0.5763006210327148},
  {'label': 'fear', 'score': 0.004816171247512102},
  {'label': 'joy', 'score': 0.0023134469520300627},
  {'label': 'neutral', 'score': 0.011871281079947948},
  {'label': 'sadness', 'score': 0.03086276538670063},
  {'label': 'surprise', 'score': 0.0024154267739504576}]]

In [18]:
# To classify if toxic
tokenizer = AutoTokenizer.from_pretrained("martin-ha/toxic-comment-model")
toxic_model = AutoModelForSequenceClassification.from_pretrained("martin-ha/toxic-comment-model")
toxic_classifier =  TextClassificationPipeline(model=toxic_model, tokenizer=tokenizer)



In [19]:
toxic_classifier(sentimentRecord)

[{'label': 'toxic', 'score': 0.9608501195907593}]

In [39]:
# For overall positive or negative
happy_tc = HappyTextClassification(model_type="DISTILBERT", "ProsusAI/finbert", num_labels=3)

05/28/2023 11:54:57 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [42]:
result = happy_tc.classify_text("Tesla's stock just decreased by 20%")
result

TextClassificationResult(label='negative', score=0.9680216908454895)